In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

In [3]:
!git clone https://github.com/XuhuiZhou/CATS.git
import sys; sys.path.append('./CATS')

Cloning into 'CATS'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 77 (delta 33), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (77/77), done.


In [4]:
import torch
from scipy.special import softmax
import numpy as np
import math
import pickle

from transformers import GPT2Tokenizer, GPT2LMHeadModel

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

In [5]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
model = GPT2LMHeadModel.from_pretrained('gpt2-medium')
model=model.to(device)

In [6]:
from datasets import load_dataset
dataset = load_dataset("wikitext", 'wikitext-2-raw-v1')

Dataset wikitext downloaded and prepared to /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/47c57a6745aa5ce8e16a5355aaa4039e3aa90d1adad87cef1ad4e0f29e74ac91. Subsequent calls will reuse this data.


In [ ]:
!python run_clm.py \
    --model_name_or_path gpt2 \
    --block_size 100\
    --dataset_name wikitext \
    --dataset_config_name wikitext-2-raw-v1 \
    --do_train \
    --do_eval \
    --output_dir /tmp/test-clm


In [ ]:
model.eval()

In [9]:
 unigram_freq = pickle.load(open('gpt-openwebtext.pickle', "rb"))
 unigram_total = sum(unigram_freq.values()) 

In [10]:
def predict(text, model, tokenizer):
    text = text
    tokenized_text = tokenizer.tokenize(text)
    sentence_score = 0
    tokens_tensor = torch.tensor([[50256] + tokenizer.convert_tokens_to_ids(tokenized_text)], device=device)
    length = len(tokenized_text)

    #unigram logprob
    uni_lp = 0.0
    for w in tokenized_text:
      uni_lp += math.log(float(unigram_freq[w])/unigram_total)

    with torch.no_grad():
        outputs = model(tokens_tensor, labels= tokens_tensor)
    loss, logits  = outputs[:2]
    sentence_score = -loss


    lp = float(loss.item()) * -1.0 * length
    penalty = ((5+length)**0.8 / (5+1)**0.8)*lp

    div_lp = -lp / uni_lp
    slor = (lp - uni_lp) / length
    pen_slor = (lp - uni_lp) / penalty

    return sentence_score, lp, penalty, div_lp, slor, pen_slor

In [78]:
def show_results():
    with open("CATS/commonsense_ability_test/{}.txt".format(test), "r") as f:
        file = f.readlines()
    num = len(file)
    count = 0
    curr = 0
    count_slor = 0
    count_pen_slor = 0

    for line in file:
        if curr < 20:
          print(line)
        line = line.strip().split("\001")
        label = int(line[0])
        score_list = []
        lp_list = []
        div_lp_list = []
        penalty_list = []
        slors_list = []
        pen_slor_list = []
        for sentence in line[1:]:
            score, lp, penalty, div_lp, slor, pen_slor = predict(sentence, model=model, tokenizer=tokenizer)
            score_list.append(score)
            lp_list.append(lp)
            div_lp_list.append(div_lp)
            penalty_list.append(penalty)
            slors_list.append(slor)
            pen_slor_list.append(pen_slor)

        ppl_list = [math.exp(-element) for element in score_list]
        if curr < 20:
            print('scores:', [element.item() for element in score_list])
            print('ppl:', ppl_list)
            print('lp:', lp_list)
            print('div_lp:', div_lp_list)
            print('penalty:', penalty_list)
            print('slor:', slors_list)
            print('pen_slor:',  pen_slor_list, '\n')

   
        predict_label = ppl_list.index(min(ppl_list))
        if predict_label==label:
            count += 1
        curr += 1

        predict_label_slore = slors_list.index(max(slors_list))
        if predict_label_slore==label:
            count_slor += 1

        predict_label_pen_slore = pen_slor_list.index(min(pen_slor_list))
        if predict_label_pen_slore==label:
            count_pen_slor += 1
        if curr <= 20:
            print('predicted label ppl:', predict_label, '  correct label:', label)
            print('correct answers:', count, '  total number of examples:', curr, '  percentage of correct answers:', count/curr, '\n')
            print('predicted label slor:', predict_label_slore, '  correct label:', label)
            print('correct answers:', count_slor, '  total number of examples:', curr, '  percentage of correct answers:', count_slor/curr, '\n')
            print('predicted label pen slor:', predict_label_pen_slore, '  correct label:', label)
            print('correct answers:', count_pen_slor, '  total number of examples:', curr, '  percentage of correct answers:', count_pen_slor/curr, '\n')


    print (' percentage of correct answers ppl:', count/num)
    print (' percentage of correct answers slor:', count_slor/num)
    print (' percentage of correct answers pen_slor:', count_pen_slor/num)

In [79]:
test = 'smr'
show_results()

1" he put an elephant into the fridge " is against common sense because elephants are usually gray while fridges are usually white . " he put an elephant into the fridge " is against common sense because an elephant is much bigger than a fridge . " he put an elephant into the fridge " is against common sense because an elephant cannot eat a fridge . 

scores: [-4.643041133880615, -4.1690897941589355, -4.5417962074279785]
ppl: [103.85971910731682, 64.65657452431266, 93.85923946129311]
lp: [-120.719069480896, -100.05815505981445, -95.37772035598755]
div_lp: [-0.5288825011342226, -0.5140723453421597, -0.5387877363199289]
penalty: [-449.10121906764556, -352.8985621036672, -308.2510865599166]
slor: [4.135924182467182, 3.9408383743832665, 3.8878615246683714]
pen_slor: [-0.23944274515084202, -0.26800936909857576, -0.26486554493350006] 

predicted label ppl: 1   correct label: 1
correct answers: 1   total number of examples: 1   percentage of correct answers: 1.0 

predicted label slor: 0  

In [80]:
test = 'ca'
show_results()

0The trophy doesn't fit into the brown suitcase because the trophy is too large.The trophy doesn't fit into the brown suitcase and the trophy is too large.

scores: [-3.7938218116760254, -3.8596951961517334]
ppl: [44.42586352534108, 47.45088595176637]
lp: [-60.701148986816406, -61.755123138427734]
div_lp: [-0.4590139699963686, -0.4783520303447394]
penalty: [-165.36795173376305, -168.2392902428603]
slor: [4.471333629466734, 4.209038605124542]
pen_slor: [-0.4326191219121281, -0.4002906668518273] 

predicted label ppl: 0   correct label: 0
correct answers: 1   total number of examples: 1   percentage of correct answers: 1.0 

predicted label slor: 0   correct label: 0
correct answers: 1   total number of examples: 1   percentage of correct answers: 1.0 

predicted label pen slor: 0   correct label: 0
correct answers: 1   total number of examples: 1   percentage of correct answers: 1.0 

1The trophy doesn't fit into the brown suitcase and the suitcase is too small.The trophy doesn't fi

In [81]:
test = 'arct_1'
show_results()

0They add a lot to the piece and I look forward to reading comments, and since comments sections are a welcome distraction from my work, Comment sections have not failedThey add a lot to the piece and I look forward to reading comments, but since comments sections always distract me from my work, Comment sections have not failed

scores: [-4.193867206573486, -4.2396368980407715]
ppl: [66.27860906351756, 69.3826542866838]
lp: [-134.20375061035156, -131.42874383926392]
div_lp: [-0.5411392037401608, -0.5365264933659271]
penalty: [-575.1816631402279, -551.0758222857645]
slor: [3.5562037134171103, 3.662371577706973]
pen_slor: [-0.197847960256695, -0.20602159325008912] 

predicted label ppl: 0   correct label: 0
correct answers: 1   total number of examples: 1   percentage of correct answers: 1.0 

predicted label slor: 1   correct label: 0
correct answers: 0   total number of examples: 1   percentage of correct answers: 0.0 

predicted label pen slor: 1   correct label: 0
correct answers:

In [82]:
test = 'arct_2'
show_results()

0They add a lot to the piece and I look forward to reading comments, and since comments sections are a welcome distraction from my work, Comment sections have not failedThey add a lot to the piece and I look forward to reading comments, but since comments sections always distract me from my work, Comment sections have not failed

scores: [-4.193867206573486, -4.2396368980407715]
ppl: [66.27860906351756, 69.3826542866838]
lp: [-134.20375061035156, -131.42874383926392]
div_lp: [-0.5411392037401608, -0.5365264933659271]
penalty: [-575.1816631402279, -551.0758222857645]
slor: [3.5562037134171103, 3.662371577706973]
pen_slor: [-0.197847960256695, -0.20602159325008912] 

predicted label ppl: 0   correct label: 0
correct answers: 1   total number of examples: 1   percentage of correct answers: 1.0 

predicted label slor: 1   correct label: 0
correct answers: 0   total number of examples: 1   percentage of correct answers: 0.0 

predicted label pen slor: 1   correct label: 0
correct answers: